In [1]:
import math
import logging
from datetime import datetime

import torch
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, models, LoggingHandler, losses, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
import numpy as np

In [2]:
import pickle

def load_process_trees(filename):
    with open(filename, 'rb') as file:
        process_trees = pickle.load(file)
    return process_trees

In [3]:
# 파일에서 불러오기
processtree_naive = load_process_trees('processtree_naive.pkl')
processtree_simple = load_process_trees('processtree_simple.pkl')
processtree_concise = load_process_trees('processtree_concise.pkl')
processtree_straightforward = load_process_trees('processtree_straightforward.pkl')
processtree_complex = load_process_trees('processtree_complex.pkl')
processtree_spaghetti = load_process_trees('processtree_spaghetti.pkl')

In [4]:
all_process_trees = []

# 각 그룹에 속한 트리들을 모두 하나의 리스트로 통합
all_process_trees.extend(processtree_naive)
all_process_trees.extend(processtree_simple)
all_process_trees.extend(processtree_concise)
all_process_trees.extend(processtree_straightforward)
all_process_trees.extend(processtree_complex)
all_process_trees.extend(processtree_spaghetti)

In [5]:
import re

def tokenize_tree(tree):
    # 특수 문자를 이스케이프하여 정규 표현식 적용 + 괄호는 넣기
    special_characters = r"[\*\+\./:;<=\?\[\]\^_`{|}~\(\)]"
    tokenized_tree = re.findall(r"\b\w+\b|" + special_characters + r"|\u2192", str(tree).replace("->", "\u2192"))
    return tokenized_tree

# Tokenize each group
tokenized_processtree_naive = [tokenize_tree(tree) for tree in processtree_naive]
tokenized_processtree_simple = [tokenize_tree(tree) for tree in processtree_simple]
tokenized_processtree_concise = [tokenize_tree(tree) for tree in processtree_concise]
tokenized_processtree_straightforward = [tokenize_tree(tree) for tree in processtree_straightforward]
tokenized_processtree_complex = [tokenize_tree(tree) for tree in processtree_complex]
tokenized_processtree_spaghetti = [tokenize_tree(tree) for tree in processtree_spaghetti]

In [6]:
all_tokenized_processtrees = []

# 토큰화된 트리들을 각 그룹에서 가져와서 통합
all_tokenized_processtrees.extend(tokenized_processtree_naive)
all_tokenized_processtrees.extend(tokenized_processtree_simple)
all_tokenized_processtrees.extend(tokenized_processtree_concise)
all_tokenized_processtrees.extend(tokenized_processtree_straightforward)
all_tokenized_processtrees.extend(tokenized_processtree_complex)
all_tokenized_processtrees.extend(tokenized_processtree_spaghetti)

In [7]:
# 전체 프로세스 트리를 100개씩 끊어 6개의 그룹에 나누기
chunk_size = 100
groups = [all_process_trees[i:i+chunk_size] for i in range(0, len(all_process_trees), chunk_size)]

# 6개의 그룹에 대해 각각 변수에 저장
processtree_naive = groups[0]
processtree_simple = groups[1]
processtree_concise = groups[2]
processtree_straightforward = groups[3]
processtree_complex = groups[4]
processtree_spaghetti = groups[5]

In [8]:
import torch
from torch.optim import Adam
from torch.nn.functional import cosine_similarity
from sentence_transformers import SentenceTransformer

In [9]:
# SBERT 모델 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sbert_model_name = 'distilbert-base-nli-mean-tokens'
sbert_model = SentenceTransformer(sbert_model_name).to(device)

In [11]:
import Levenshtein

## Levenshtein similarity fine-tuning

In [ ]:
## (tree1, tree2, levenshtein sim)
from sentence_transformers.readers import InputExample

def make_dataset(pts):

    data = []
    total_iterations = len(pts) * (len(pts) + 1) // 2  # 전체 반복 횟수 계산

    for i in range(len(pts)):
        for j in range(i, len(pts)):
            seq1 = pts[i]
            seq2 = pts[j]
            distance = Levenshtein.distance(seq1, seq2)
            similarity = 1 - distance / max(len(seq1), len(seq2))
            new_data = InputExample(texts=[seq1, seq2], label=similarity)
            data.append(new_data)

    return data

datasets = make_dataset(all_process_trees)


In [ ]:
import random

# 데이터셋의 크기와 1퍼센트에 해당하는 샘플 개수 계산
total_samples = len(datasets)
one_percent_samples = int(total_samples * 0.1)

# 무작위로 1퍼센트의 샘플 선택
random_samples = random.sample(datasets, one_percent_samples)

# 선택된 샘플 확인
print("선택된 샘플 수:", len(random_samples))

선택된 샘플 수: 18030


In [ ]:
from sklearn.model_selection import train_test_split

# 데이터셋을 train과 나머지로 나눔
train_data, temp_data = train_test_split(random_samples, test_size=0.2, random_state=42)

# 나머지를 test와 eval로 나눔 (전체 데이터셋의 20%를 각각의 test와 eval로 사용)
test_data, eval_data = train_test_split(temp_data, test_size=0.5, random_state=42)

In [ ]:
# ## (tree1_encoding, tree2_encoding, levenshtine sim)
# import torch
# from tqdm import tqdm

# def make_dataset(pts, model):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)

#     data = []
#     total_iterations = len(pts) * (len(pts) + 1) // 2  # 전체 반복 횟수 계산
#     progress_bar = tqdm(total=total_iterations, desc="Processing data", leave=False)

#     for i in range(len(pts)):
#         for j in range(i, len(pts)):
#             seq1 = pts[i]
#             seq2 = pts[j]
#             distance = Levenshtein.distance(seq1, seq2)
#             similarity = 1 - distance / max(len(seq1), len(seq2))
#             encoding1 = model.encode(seq1)
#             encoding2 = model.encode(seq2)
#             new_data = [encoding1, encoding2, similarity]
#             data.append(new_data)

#             # tqdm 업데이트
#             progress_bar.update()

#     progress_bar.close()
#     return data

# datasets = make_dataset(all_process_trees, sbert_model)


Processing data:  46%|████▋     | 83687/180300 [20:29<23:03, 69.82it/s]Exception ignored in: <function tqdm.__del__ at 0x7b8ea3f6aef0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/usr/local/lib/python3.10/dist-packages/tqdm/notebook.py", line 273, in close
    if self.disable:
AttributeError: 'tqdm_notebook' object has no attribute 'disable'


In [ ]:
# import pickle

# # 데이터를 파일에 저장
# with open("datasets.pkl", "wb") as f:
#     pickle.dump(datasets, f)

# # 저장된 파일을 로드
# with open("datasets.pkl", "rb") as f:
#     loaded_datasets = pickle.load(f)

In [ ]:
from torch.utils.data import DataLoader
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

# Train Dataloader
train_dataloader = DataLoader(
    train_data,
    shuffle=True,
    batch_size=32,
)

# Evaluator by sts-validation
dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    eval_data,
    name="sts-dev",
)

# Evaluator by sts-test
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    test_data,
    name="sts-test",
)

In [ ]:
from sentence_transformers import SentenceTransformer, models

# Load Embedding Model
embedding_model = models.Transformer(
    model_name_or_path="klue/roberta-base",
    max_seq_length=256,
    do_lower_case=True
)

# Only use Mean Pooling -> Pooling all token embedding vectors of sentence.
pooling_model = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

model = SentenceTransformer(modules=[embedding_model, pooling_model])

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Use CosineSimilarityLoss
train_loss = losses.CosineSimilarityLoss(model=model)
model.to(device)

# warmup steps
warmup_steps = math.ceil(len(datasets) * 4 / 32 * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

# Training
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=dev_evaluator,
    epochs=4,
    evaluation_steps=int(len(train_dataloader)*0.1),
    warmup_steps=warmup_steps
)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/451 [00:00<?, ?it/s]

Iteration:   0%|          | 0/451 [00:00<?, ?it/s]

Iteration:   0%|          | 0/451 [00:00<?, ?it/s]

Iteration:   0%|          | 0/451 [00:00<?, ?it/s]

In [ ]:
def embed_processes_with_sbert(tokenized_processes):

    # 임베딩을 저장할 리스트 초기화
    sbert_embeddings = []

    # tokenized_processes에 대한 반복문
    for tree_tokens in tokenized_processes:
        # 프로세스 트리를 텍스트로 펼치고 토큰화
        process_tree_text = " ".join(tree_tokens)

        # SBERT 모델을 사용하여 문장 임베딩 얻기
        sbert_embedding = model.encode([process_tree_text])

        # 평균 풀링
        average_pooled_embedding = np.mean(sbert_embedding, axis=0)
        sbert_embeddings.append(average_pooled_embedding)

    # 모든 프로세스 트리에 대한 임베딩이 sbert_embeddings에 저장됨
    return sbert_embeddings

In [ ]:
groups = [tokenized_processtree_naive, tokenized_processtree_simple, tokenized_processtree_concise, tokenized_processtree_straightforward, tokenized_processtree_complex, tokenized_processtree_spaghetti]

SSL_sbert_embeddings_naive = []
SSL_sbert_embeddings_simple = []
SSL_sbert_embeddings_concise = []
SSL_sbert_embeddings_straightforward = []
SSL_sbert_embeddings_complex = []
SSL_sbert_embeddings_spaghetti = []

for idx, group in enumerate(groups):
    sbert_embeddings = embed_processes_with_sbert(group)

    # 그룹별로 결과를 저장
    if idx == 0:
        SSL_sbert_embeddings_naive = sbert_embeddings
    elif idx == 1:
        SSL_sbert_embeddings_simple = sbert_embeddings
    elif idx == 2:
        SSL_sbert_embeddings_concise = sbert_embeddings
    elif idx == 3:
        SSL_sbert_embeddings_straightforward = sbert_embeddings
    elif idx == 4:
        SSL_sbert_embeddings_complex = sbert_embeddings
    elif idx == 5:
        SSL_sbert_embeddings_spaghetti = sbert_embeddings

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def cosine_avg(embeddings_groups):
    num_groups = len(embeddings_groups)
    similarity_matrix = np.zeros((num_groups, num_groups))

    for i in range(num_groups):
        for j in range(num_groups):
            embeddings1 = embeddings_groups[i]
            embeddings2 = embeddings_groups[j]

            similarity_matrix[i, j] = np.mean(cosine_similarity(embeddings1, embeddings2))

    # Print the similarity matrix
    upper_triangular_matrix = np.triu(similarity_matrix)
    print(upper_triangular_matrix)

In [ ]:
embeddings_groups_sbert = [SSL_sbert_embeddings_naive, SSL_sbert_embeddings_simple, SSL_sbert_embeddings_concise, SSL_sbert_embeddings_straightforward, SSL_sbert_embeddings_complex, SSL_sbert_embeddings_spaghetti]
cosine_avg(embeddings_groups_sbert)
# 18000개

[[0.77634996 0.70031404 0.62903994 0.30305019 0.29385933 0.27642447]
 [0.         0.68837893 0.63567698 0.28265548 0.28275436 0.26828846]
 [0.         0.         0.65251207 0.25199154 0.25468501 0.26694867]
 [0.         0.         0.         0.79331201 0.74621296 0.69394815]
 [0.         0.         0.         0.         0.73559183 0.68871778]
 [0.         0.         0.         0.         0.         0.70876926]]


In [ ]:
embeddings_groups_sbert = [SSL_sbert_embeddings_naive, SSL_sbert_embeddings_simple, SSL_sbert_embeddings_concise, SSL_sbert_embeddings_straightforward, SSL_sbert_embeddings_complex, SSL_sbert_embeddings_spaghetti]
cosine_avg(embeddings_groups_sbert)
# 1800개

[[0.86109763 0.77198547 0.68093139 0.29826051 0.25707501 0.23810615]
 [0.         0.7805711  0.7076239  0.24495715 0.29047441 0.25249526]
 [0.         0.         0.77330655 0.19204724 0.23413037 0.31313813]
 [0.         0.         0.         0.81391525 0.7189579  0.64738327]
 [0.         0.         0.         0.         0.77254045 0.66988492]
 [0.         0.         0.         0.         0.         0.79755712]]


## GED similarity fine-tuning

In [10]:
import networkx as nx
import matplotlib.pyplot as plt
from zss import simple_distance, distance
from zss import Node

class TreeNode:
    def __init__(self, value):
        self.value = value
        self.children = []

    def add_child(self, child):
        self.children.append(child)

    def print_tree(self, depth=0):
        print("  " * depth + self.value)
        for child in self.children:
            child.print_tree(depth + 1)

    def to_zss_node(self):
        if not self.children:
            return Node(self.value, [])
        else:
            children_nodes = [child.to_zss_node() for child in self.children]
            return Node(self.value, children_nodes)

    def count_nodes(self):
        # 현재 노드를 포함하여 노드 수 계산
        count = 1
        for child in self.children:
            count += child.count_nodes()
        return count

    def count_edges(self):
        # 현재 노드의 자식 수를 엣지로 간주하여 엣지 수 계산
        edge_count = len(self.children)
        for child in self.children:
            edge_count += child.count_edges()
        return edge_count



In [11]:
def text_to_tree(sentence):
    stack = []
    root = TreeNode(None)
    sentence = sentence.replace(" ", "")
    for i in range(len(sentence)):
        if sentence[i] != '(' and sentence[i] != ')':
            cur = sentence[i]
            child = TreeNode(cur)
            root.add_child(child)
        elif sentence[i] == '(':
            stack.append(child)
            root = child
        else:
            stack.pop()
            if stack != []:
                root = stack[-1]
    # tree = root.to_zss_node()
    return root

In [12]:
def tree_edit_distance(graph1, graph2):
    tree1 = graph1.to_zss_node()
    tree2 = graph2.to_zss_node()
    dist = simple_distance(tree1, tree2)
    return dist

In [13]:
def max_edit_distance(graph1, graph2):
    max1 = graph1.count_nodes() + graph1.count_edges()
    max2 = graph2.count_nodes() + graph2.count_edges()
    dist = max1 + max2
    return dist

In [14]:
def graph_edit_similarity(sentence1, sentence2):
    graph1 = text_to_tree(sentence1)
    graph2 = text_to_tree(sentence2)

    ged = tree_edit_distance(graph1, graph2)
    maxged = max_edit_distance(graph1, graph2)

    sim = 1 - (ged / maxged)

    return sim

In [15]:
def ged_average_similarity(tree_group1, tree_group2):
    total_similarity = 0.0

    for tree1 in tree_group1:
        for tree2 in tree_group2:
            similarity = graph_edit_similarity(tree1, tree2)
            total_similarity += similarity

    # 프로세스 트리 쌍의 총 수
    total_pairs = len(tree_group1) * len(tree_group2)

    # 전체 유사도의 평균
    average_similarity = total_similarity / total_pairs

    return average_similarity

In [18]:
from tqdm import tqdm

def make_dataset(pts):
    data = []
    total_iterations = len(pts) * (len(pts) + 1) // 2
    progress_bar = tqdm(total=total_iterations, desc='Progress', unit=' pairs')

    for i in range(len(pts)):
        for j in range(i, len(pts)):
            seq1 = pts[i]
            seq2 = pts[j]
            similarity = graph_edit_similarity(seq1, seq2)
            new_data = InputExample(texts=[seq1, seq2], label=similarity)
            data.append(new_data)
            progress_bar.update(1)  # 반복마다 진행률 업데이트

    progress_bar.close()
    return data

datasets = make_dataset(all_process_trees)

Progress: 100%|██████████| 180300/180300 [3:56:49<00:00, 12.69 pairs/s]  


In [25]:
import pickle

# 데이터셋을 파일로 저장
with open("datasets.pkl", "wb") as f:
    pickle.dump(datasets, f)

In [16]:
datasets = load_process_trees('datasets.pkl')

In [17]:
groups = [processtree_naive, processtree_simple, processtree_concise, processtree_straightforward, processtree_complex,processtree_spaghetti]

# Initialize a 6x6 matrix for similarities
similarity_matrix = np.zeros((6, 6))

# Calculate and update the matrix with average similarity for each pair of groups
for i, group1 in enumerate(groups):
    for j, group2 in enumerate(groups):
        average_similarity = ged_average_similarity(group1, group2)

        # Update the matrix with similarity values (both symmetric positions)
        similarity_matrix[i][j] = average_similarity

# Print the similarity matrix
print("Similarity Matrix:")
print(similarity_matrix)

KeyboardInterrupt: 

In [18]:
import random

# 데이터셋의 크기와 1퍼센트에 해당하는 샘플 개수 계산
total_samples = len(datasets)
one_percent_samples = int(total_samples * 0.01)

# 무작위로 1퍼센트의 샘플 선택
random_samples = random.sample(datasets, one_percent_samples)

# 선택된 샘플 확인
print("선택된 샘플 수:", len(random_samples))

선택된 샘플 수: 1803


In [19]:
from sklearn.model_selection import train_test_split

# 데이터셋을 train과 나머지로 나눔
train_data, temp_data = train_test_split(random_samples, test_size=0.2, random_state=42)

# 나머지를 test와 eval로 나눔 (전체 데이터셋의 20%를 각각의 test와 eval로 사용)
test_data, eval_data = train_test_split(temp_data, test_size=0.5, random_state=42)

In [20]:
from torch.utils.data import DataLoader
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

# Train Dataloader
train_dataloader = DataLoader(
    train_data,
    shuffle=True,
    batch_size=32,
)

# Evaluator by sts-validation
dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    eval_data,
    name="sts-dev",
)

# Evaluator by sts-test
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    test_data,
    name="sts-test",
)

In [21]:
from sentence_transformers import SentenceTransformer, models

# Load Embedding Model
embedding_model = models.Transformer(
    model_name_or_path="klue/roberta-base",
    max_seq_length=256,
    do_lower_case=True
)

# Only use Mean Pooling -> Pooling all token embedding vectors of sentence.
pooling_model = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

model = SentenceTransformer(modules=[embedding_model, pooling_model])

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# Use CosineSimilarityLoss
train_loss = losses.CosineSimilarityLoss(model=model)
model.to(device)

# warmup steps
warmup_steps = math.ceil(len(datasets) * 4 / 32 * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

# Training
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=dev_evaluator,
    epochs=4,
    evaluation_steps=int(len(train_dataloader)*0.1),
    warmup_steps=warmup_steps
)

RuntimeError: Failed to import transformers.optimization because of the following error (look up to see its traceback):
cannot import name 'LRScheduler' from 'torch.optim.lr_scheduler' (c:\Users\82105\anaconda3\envs\PTE\lib\site-packages\torch\optim\lr_scheduler.py)

In [ ]:
groups = [tokenized_processtree_naive, tokenized_processtree_simple, tokenized_processtree_concise, tokenized_processtree_straightforward, tokenized_processtree_complex, tokenized_processtree_spaghetti]

SSL_sbert_embeddings_naive = []
SSL_sbert_embeddings_simple = []
SSL_sbert_embeddings_concise = []
SSL_sbert_embeddings_straightforward = []
SSL_sbert_embeddings_complex = []
SSL_sbert_embeddings_spaghetti = []

for idx, group in enumerate(groups):
    sbert_embeddings = embed_processes_with_sbert(group)

    # 그룹별로 결과를 저장
    if idx == 0:
        SSL_sbert_embeddings_naive = sbert_embeddings
    elif idx == 1:
        SSL_sbert_embeddings_simple = sbert_embeddings
    elif idx == 2:
        SSL_sbert_embeddings_concise = sbert_embeddings
    elif idx == 3:
        SSL_sbert_embeddings_straightforward = sbert_embeddings
    elif idx == 4:
        SSL_sbert_embeddings_complex = sbert_embeddings
    elif idx == 5:
        SSL_sbert_embeddings_spaghetti = sbert_embeddings

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def cosine_avg(embeddings_groups):
    num_groups = len(embeddings_groups)
    similarity_matrix = np.zeros((num_groups, num_groups))

    for i in range(num_groups):
        for j in range(num_groups):
            embeddings1 = embeddings_groups[i]
            embeddings2 = embeddings_groups[j]

            similarity_matrix[i, j] = np.mean(cosine_similarity(embeddings1, embeddings2))

    # Print the similarity matrix
    upper_triangular_matrix = np.triu(similarity_matrix)
    print(upper_triangular_matrix)

In [ ]:
embeddings_groups_sbert = [SSL_sbert_embeddings_naive, SSL_sbert_embeddings_simple, SSL_sbert_embeddings_concise, SSL_sbert_embeddings_straightforward, SSL_sbert_embeddings_complex, SSL_sbert_embeddings_spaghetti]
cosine_avg(embeddings_groups_sbert)
# 18000개

[[0.80669165 0.77053416 0.70179319 0.81980646 0.79411203 0.75778657]
 [0.         0.77959275 0.73616874 0.80601615 0.80257154 0.77923   ]
 [0.         0.         0.75148857 0.75938213 0.76467168 0.7758199 ]
 [0.         0.         0.         0.87225968 0.8441366  0.80565047]
 [0.         0.         0.         0.         0.8422206  0.81215394]
 [0.         0.         0.         0.         0.         0.82652491]]


In [ ]:
embeddings_groups_sbert = [SSL_sbert_embeddings_naive, SSL_sbert_embeddings_simple, SSL_sbert_embeddings_concise, SSL_sbert_embeddings_straightforward, SSL_sbert_embeddings_complex, SSL_sbert_embeddings_spaghetti]
cosine_avg(embeddings_groups_sbert)
# 1800개

# ---

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch

class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        pair, score = self.data[idx][:2], self.data[idx][2]
        return pair[0], pair[1], score


# 데이터셋 인스턴스 생성
dataset = MyDataset(datasets)

# 데이터로더 생성
batch_size = 128
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
from torch.optim import Adam

# optimizer 정의
optimizer = Adam(sbert_model.parameters(), lr=1e-5)

# 학습 함수 정의
def train_epoch(model, data_loader, optimizer):
    model.train()
    total_loss = 0

    # tqdm을 사용하여 진행 상황 표시
    progress_bar = tqdm(data_loader, desc="Training", leave=False)

    for encodings1, encodings2, scores in progress_bar:
        optimizer.zero_grad()
        output1 = model(**encodings1)
        output2 = model(**ncodings2)
        embeddings1 = output1.pooler_output
        embeddings2 = output2.pooler_output
        loss = 1 - cosine_similarity(embeddings1, embeddings2).mean()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # tqdm 업데이트
        progress_bar.set_postfix({'Loss': loss.item()})

    progress_bar.close()
    return total_loss / len(data_loader)

# 학습 반복
num_epochs = 10
for epoch in range(num_epochs):
    loss = train_epoch(sbert_model, data_loader, optimizer)
    print(f"Epoch {epoch+1}, Loss: {loss}")


TypeError: SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
) argument after ** must be a mapping, not Tensor